In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    # ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD" and attack_name!= "DF":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

unique_labels, counts = np.unique(y_label_0_1, return_counts=True)
print(unique_labels, counts)
valid_labels = unique_labels[counts > 100]
mask = np.isin(y_label_0_1, valid_labels)
x_test_adv_0_1 = x_test_adv_0_1[mask]
y_label_0_1 = y_label_0_1[mask]
print(np.unique(y_label_0_1, return_counts = True))


[ 1  2  3  4  5  6  8 10] [954961     14 238757      4     20 238756      1      5]
(array([1, 3, 6]), array([954961, 238757, 238756]))


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '3' encode 1
defense '6' encode 2


In [5]:
# Collect 1% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_1, _ , y_1 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.01, stratify=y_label_0_1, random_state=42)


In [6]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt1 = DecisionTreeClassifier()
# dt1.fit(X_1, y_1)

xgb
RF


RandomForestClassifier()

In [7]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude3Attack/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude3Attack/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[6] [238753]
(238753, 42) (238753,)
Save 6 to baseline_Def6.npy
Execution Time: 0.133847 seconds
(716259,)
[1 3 6] [457003  20562 238694]
(457003, 42) (457003,)
Save 1 to BIM_Def1.npy
(20562, 42) (20562,)
Save 3 to BIM_Def3.npy
(238694, 42) (238694,)
Save 6 to BIM_Def6.npy
Execution Time: 0.320414 seconds
(716259,)
[1 3 6] [476771    800 238688]
(476771, 42) (476771,)
Save 1 to FGSM_Def1.npy
(800, 42) (800,)
Save 3 to FGSM_Def3.npy
(238688, 42) (238688,)
Save 6 to FGSM_Def6.npy
Execution Time: 0.310125 seconds
(716259,)
[1 3 6] [457003  20562 238694]
(457003, 42) (457003,)
Save 1 to PGD_Def1.npy
(20562, 42) (20562,)
Save 3 to PGD_Def3.npy
(238694, 42) (238694,)
Save 6 to PGD_Def6.npy
Execution Time: 0.301010 seconds
(716259,)
[1 3 6] [707057    927   8275]
(707057, 42) (707057,)
Save 1 to DF_Def1.npy
(927, 42) (927,)
Save 3 to DF_Def3.npy
(8275, 42) (8275,)
Save 6 to DF_Def6.npy
Execution Time: 0.299683 seconds
(716259,)
[1 3 6] [399948  73741 242570]
(399948, 42) (399948,)
S

In [8]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude3Attack/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude3Attack/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[6] [238753]
(238753, 42) (238753,)
Save 6 to baseline_Def6.npy
Execution Time: 0.627468 seconds
(716259,)
[1 3 6] [464775  12731 238753]
(464775, 42) (464775,)
Save 1 to BIM_Def1.npy
(12731, 42) (12731,)
Save 3 to BIM_Def3.npy
(238753, 42) (238753,)
Save 6 to BIM_Def6.npy
Execution Time: 1.333158 seconds
(716259,)
[1 3 6] [476995    509 238755]
(476995, 42) (476995,)
Save 1 to FGSM_Def1.npy
(509, 42) (509,)
Save 3 to FGSM_Def3.npy
(238755, 42) (238755,)
Save 6 to FGSM_Def6.npy
Execution Time: 1.429693 seconds
(716259,)
[1 3 6] [464775  12731 238753]
(464775, 42) (464775,)
Save 1 to PGD_Def1.npy
(12731, 42) (12731,)
Save 3 to PGD_Def3.npy
(238753, 42) (238753,)
Save 6 to PGD_Def6.npy
Execution Time: 1.606725 seconds
(716259,)
[1 3 6] [716202      7     50]
(716202, 42) (716202,)
Save 1 to DF_Def1.npy
(7, 42) (7,)
Save 3 to DF_Def3.npy
(50, 42) (50,)
Save 6 to DF_Def6.npy
Execution Time: 1.634126 seconds
(716259,)
[1 3 6] [451636  24768 239855]
(451636, 42) (451636,)
Save 1 to

In [9]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt1.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude3Attack/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude3Attack/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [10]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_Exclude3Attack/WUSTL_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [11]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_Exclude3Attack.csv")

In [12]:
all_attack_list

0       1   2       3   4   5       6   7   8   9   10  11      12
0   XGB1       0   0       0   0   0  238753   0   0   0   0   0  238753
1   XGB1  457003   0   20562   0   0  238694   0   0   0   0   0  716259
2   XGB1  476771   0     800   0   0  238688   0   0   0   0   0  716259
3   XGB1  457003   0   20562   0   0  238694   0   0   0   0   0  716259
4   XGB1  707057   0     927   0   0    8275   0   0   0   0   0  716259
5   XGB1  399948   0   73741   0   0  242570   0   0   0   0   0  716259
6   XGB1       0   0       0   0   0  716259   0   0   0   0   0  716259
7   XGB1  150783   0    4994   0   0  560482   0   0   0   0   0  716259
8   XGB1    9768   0  467801   0   0  238690   0   0   0   0   0  716259
9   XGB1  463965   0   13587   0   0  238707   0   0   0   0   0  716259
10   RF1       0   0       0   0   0  238753   0   0   0   0   0  238753
11   RF1  464775   0   12731   0   0  238753   0   0   0   0   0  716259
12   RF1  476995   0     509   0   0  238755   0   0   0   0   0  716259
13   RF1  464775   0   12731   0   0  238753   0   0   0   0   0  716259
14   RF1  716202   0       7   0   0      50   0   0   0   0   0  716259
15   RF1  451636   0   24768   0   0  239855   0   0   0   0   0  716259
16   RF1       0   0       0   0   0  716259   0   0   0   0   0  716259
17   RF1  198884   0       0   0   0  517375   0   0   0   0   0  716259
18   RF1   20652   0  456854   0   0  238753   0   0   0   0   0  716259
19   RF1  472226   0    5250   0   0  238783   0   0   0   0   0  716259
20   DT1       0   0       0   0   0       0   0   0   0   0   0       0
21   DT1       0   0       0   0   0       0   0   0   0   0   0       0
22   DT1       0   0       0   0   0       0   0   0   0   0   0       0
23   DT1       0   0       0   0   0       0   0   0   0   0   0       0
24   DT1       0   0       0   0   0       0   0   0   0   0   0       0
25   DT1       0   0       0   0   0       0   0   0   0   0   0       0
26   DT1       0   0       0   0   0       0   0   0   0   0   0       0
27   DT1       0   0       0   0   0       0   0   0   0   0   0       0
28   DT1       0   0       0   0   0       0   0   0   0   0   0       0
29   DT1       0   0       0   0   0       0   0   0   0   0   0       0